In [1]:
# Libraries use 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh
import geopandas as gpd
from pyproj import CRS
from helper import create_dummy_df

%matplotlib inline

In [2]:
# Read dataset
df = pd.read_csv('../NYC/data/sample.csv', low_memory=False, verbose=1)
df.head()

Tokenization took: 40.86 ms
Type conversion took: 21.97 ms
Parser memory cleanup took: 1.96 ms


,neighbourhood,neighbourhood_group,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,availability_365,number_of_reviews
0,Washington Heights,Manhattan,Other,Private room,2,1.0,1.0,2.0,100.0,271.977649,70.367565,1,365.000000,1.0
1,Manhattan,Manhattan,Apartment,Entire home/apt,2,1.0,1.0,1.0,225.0,350.000000,95.000000,1,335.000000,48.0
2,Brooklyn,Brooklyn,Guest suite,Entire home/apt,3,1.0,1.0,3.0,89.0,500.000000,70.367565,1,276.000000,322.0
3,Midtown East,Manhattan,Apartment,Entire home/apt,2,1.0,1.0,1.0,200.0,300.000000,125.000000,2,213.392858,78.0
4,Brooklyn,Brooklyn,Apartment,Private room,2,1.0,1.0,1.0,60.0,450.000000,74.432257,1,365.000000,50.0


In [3]:
df.isnull().sum()

neighbourhood          0
neighbourhood_group    0
property_type          0
room_type              0
accommodates           0
bathrooms              0
bedrooms               0
beds                   0
price                  0
security_deposit       0
cleaning_fee           0
guests_included        0
availability_365       0
number_of_reviews      0
dtype: int64

In [4]:
df.describe()

,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,availability_365,number_of_reviews
count,49487.000000,49487.000000,49487.000000,49487.000000,49487.000000,49487.000000,49487.000000,49487.000000,49487.000000,49487.000000
mean,2.836139,1.151232,1.278295,1.567563,162.753430,369.787693,74.432257,1.500637,213.392858,30.939590
std,1.875410,0.432080,0.673175,1.102365,419.474259,390.858548,56.318910,1.154617,96.366416,46.473046
min,1.000000,0.500000,1.000000,1.000000,10.000000,95.000000,5.000000,1.000000,1.000000,1.000000
25%,2.000000,1.000000,1.000000,1.000000,68.000000,271.977649,40.000000,1.000000,175.000000,4.000000
50%,2.000000,1.000000,1.000000,1.000000,101.000000,271.977649,70.367565,1.000000,213.392858,20.000000
75%,4.000000,1.000000,1.000000,2.000000,175.000000,369.787693,80.000000,2.000000,267.000000,30.939590
max,22.000000,15.500000,21.000000,40.000000,10000.000000,5100.000000,1000.000000,16.000000,365.000000,746.000000


In [5]:
df.dtypes

neighbourhood           object
neighbourhood_group     object
property_type           object
room_type               object
accommodates             int64
bathrooms              float64
bedrooms               float64
beds                   float64
price                  float64
security_deposit       float64
cleaning_fee           float64
guests_included          int64
availability_365       float64
number_of_reviews      float64
dtype: object

In [6]:
# List of column names of categorical variables
cat_df = df.select_dtypes(include=['object'])
cat_cols_list = cat_df.columns

df_new = create_dummy_df(df, cat_cols_list, dummy_na=False)
df_new.shape

(49487, 251)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

X = df_new.drop(['price'], axis=1)
y = df_new['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=30)
lm_model=LinearRegression()
lm_model.fit(X_train, y_train)

# Predict and score the model
y_test_preds = lm_model.predict(X_test)
y_train_preds = lm_model.predict(X_train)

#Scoring model
test_score = r2_score(y_test, y_test_preds)
train_score = r2_score(y_train, y_train_preds)

print("The r-squared on the training data was {}.  The r-squared on the test data was {}.".format(train_score, test_score))

The r-squared on the training data was 0.15842032486161883.  The r-squared on the test data was 0.17540873724225736.
